In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import pickle

# Function to preprocess images
def preprocess_image(image_path, target_size):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found or unable to load: {image_path}")
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Load pretrained ResNet50 model and remove the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
feature_extractor = Model(inputs=base_model.input, outputs=x)

# Extract features from a list of images
def extract_features(image_paths, model, target_size=(224, 224)):
    features = []
    for image_path in image_paths:
        try:
            img = preprocess_image(image_path, target_size)
            feature = model.predict(img)
            features.append(feature.flatten())
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    return np.array(features)

# Function to recursively find image files in a directory
def find_image_files(directory, extensions=('.png', '.jpg', '.jpeg')):
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Directory containing ear images
ear_image_directory = r"D:\Mini_Project\Ear dataset"

# Verify the directory exists
if not os.path.exists(ear_image_directory):
    raise FileNotFoundError(f"The directory {ear_image_directory} does not exist.")

# Generate a list of file paths for all images in the directory and subdirectories
ear_image_paths = find_image_files(ear_image_directory)

# Check if any images are found
if len(ear_image_paths) == 0:
    raise FileNotFoundError(f"No images found in the directory {ear_image_directory} with the specified extensions.")

# Extract features from the dataset
ear_features = extract_features(ear_image_paths, feature_extractor)

# Check if ear_features is empty
if ear_features.size == 0:
    raise ValueError("No features extracted from the ear images. Check the image paths and preprocessing steps.")

# Save the extracted features and image paths
with open('ear_features.pkl', 'wb') as f:
    pickle.dump((ear_features, ear_image_paths), f)
print("Features extracted and saved.")




1/1 [==============================] - 1s 513ms/step
Features extracted and saved.


In [ ]:
# Load the precomputed features and image paths
with open('ear_features.pkl', 'rb') as f:
    ear_features, ear_image_paths = pickle.load(f)
print("Features and paths loaded.")

# Find the 5 most similar ear images
def find_similar_ears(input_image_path, ear_features, ear_image_paths, model, top_n=5):
    input_image = preprocess_image(input_image_path, target_size=(224, 224))
    input_feature = model.predict(input_image).flatten().reshape(1, -1)
    similarities = cosine_similarity(input_feature, ear_features)
    similar_indices = np.argsort(similarities[0])[-top_n:]
    similar_images = [cv2.imread(ear_image_paths[i]) for i in similar_indices[::-1]]  # Sort in descending order
    similarity_scores = similarities[0][similar_indices[::-1]]
    return list(zip(similar_images, similarity_scores))

# Input ear image path
input_image_path = r"D:\Mini_Project\IMG-20230212-WA0021.jpg"
similar_ears = find_similar_ears(input_image_path, ear_features, ear_image_paths, feature_extractor)
print("5 most similar ears found.")

# Display the similar ear images and their similarity scores
for i, (similar_ear, score) in enumerate(similar_ears):
    cv2.imshow(f'Similar Ear {i+1} - Similarity: {score:.4f}', similar_ear)
    cv2.waitKey(0)  # Wait for a key press to display the next image

cv2.destroyAllWindows()

Features and paths loaded.
1/1 [==============================] - 0s 204ms/step
5 most similar ears found.
